In [98]:
import os;
import json;
import builtins
import pandas as pd;
import duckdb

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
threads = ["1", "4", "16", "32"]
op = ['join', 'merge']
datasets = ['fb', 'wiki', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal', 'osm', 'books']
test_results = []
for op in op:
    for dataset in datasets:
        for thread in threads:
            testdir = os.path.join(dir, "_".join([op, dataset, thread]), "outputs", "results")
            if (not os.path.exists(testdir)):
                continue
            rundirs = os.listdir(testdir)
            for rundir in rundirs:
                for test_result_file in os.listdir(os.path.join(testdir, rundir)):
                    json_file = builtins.open(os.path.join(testdir, rundir, test_result_file))
                    test_result = json.load(json_file)
                    test_result['run'] = rundir
                    test_result['dataset'] = dataset
                    test_result['op'] = op
                    if dataset == 'osm' or dataset == 'books':
                        test_result['input_size'] = 800_000_000 
                    else:
                        test_result['input_size'] = 200_000_000 
                    test_results.append(test_result)
                    json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

Index(['command', 'run', 'dataset', 'op', 'input_size', 'result.checksum',
       'result.duration_ns', 'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


In [99]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "btree256": 256,
    "btree512": 512,
    "btree1024": 1024,
    "btree1048": 1024,
    "btree2048": 2048,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
indexType = {
    "pgm64": "PGM",
    "pgm128": "PGM",
    "pgm256": "PGM",
    "pgm512": "PGM",
    "pgm1024": "PGM",
    "pgm2048": "PGM",
    "btree256": "BTREE",
    "btree512": "BTREE",
    "btree1024": "BTREE",
    "btree1048": "BTREE",
    "btree2048": "BTREE",
    "hashjoin": "NA",
    "btree": 0,
    "sj": "SJ",
    "sj2": "NA",
    "standard_merge": "SM"
};
dataset_type = {
    "fb": "REAL",
    "wiki": "REAL",
    "osm": "REAL",
    "books": "REAL",
    "uniform_dense": "SYNTHETIC",
    "uniform_sparse": "SYNTHETIC",
    "normal": "SYNTHETIC",
    "lognormal": "SYNTHETIC",
};
dataset_display = {
    "fb": "fb",
    "wiki": "wiki",
    "osm": "osm",
    "books": "books",
    "uniform_dense": "udense",
    "uniform_sparse": "usparse",
    "normal": "normal",
    "lognormal": "lognormal",
};
dataset_order = {
    "fb": 0,
    "wiki": 1,
    "osm": 2,
    "books": 3,
    "uniform_dense": 4,
    "uniform_sparse": 5,
    "normal": 6,
    "lognormal": 7,
};

test_dataframe["threads"] = test_dataframe["spec.num_threads"]
test_dataframe["duration_sec"] = test_dataframe["result.duration_ns"] / (1000000000)
test_dataframe["ratio"] = test_dataframe["spec.common_key"]
test_dataframe["algo"] = test_dataframe["spec.algo_name"]
test_dataframe["inner_bytes_fetched"] = test_dataframe["result.inner_total_bytes_fetched"]
test_dataframe["epsilon"] = test_dataframe["spec.algo_name"].map(lambda x: epsilon[x.lower()])
test_dataframe["index_type"] = test_dataframe["spec.algo_name"].map(lambda x: indexType[x.lower()])
test_dataframe["index_name"] = test_dataframe["spec.algo_name"]
test_dataframe["dataset_type"] = test_dataframe["dataset"].map(lambda x: dataset_type[x.lower()])

results = test_dataframe[["dataset", "input_size", "dataset_type", "threads", "ratio", "op", "algo", "index_name", "index_type", "epsilon", "duration_sec", "inner_bytes_fetched"]]
results


,dataset,input_size,dataset_type,threads,ratio,op,algo,index_name,index_type,epsilon,duration_sec,inner_bytes_fetched
0,fb,200000000,REAL,1,50,join,pgm2048,pgm2048,PGM,2048,0.929766,3199258768
1,fb,200000000,REAL,1,30,join,pgm256,pgm256,PGM,256,1.404009,3183752272
2,fb,200000000,REAL,1,70,join,btree2048,btree2048,BTREE,2048,1.038459,3200000000
3,fb,200000000,REAL,1,80,join,hashJoin,hashJoin,NA,0,36.665862,3200000000
4,fb,200000000,REAL,1,70,join,hashJoin,hashJoin,NA,0,39.843666,3200000000
...,...,...,...,...,...,...,...,...,...,...,...,...
5515,books,800000000,REAL,32,100,merge,btree256,btree256,BTREE,256,10.590426,12800065184
5516,books,800000000,REAL,32,50,merge,btree1024,btree1024,BTREE,1024,10.417381,12800062688
5517,books,800000000,REAL,32,100,merge,pgm2048,pgm2048,PGM,2048,10.289159,12800065184
5518,books,800000000,REAL,32,50,merge,pgm1024,pgm1024,PGM,1024,10.988311,12800062688


In [ ]:
rows = duckdb.sql(
    " SELECT ratio, epsilon, op, index_type, AVG(duration_sec) as v FROM results "
    " WHERE threads = 1 AND input_size=200000000 AND op='join' AND index_type!='NA'"
    " GROUP BY ratio, threads, op, input_size, index_type, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
p = (duckdb.sql("PIVOT rows ON index_type,epsilon USING avg(v) GROUP BY ratio ORDER BY RATIO"))
print(p)
print(p.df()[['ratio', 'SJ_0', 'BTREE_256', 'BTREE_2048', 'PGM_256', 'PGM_2048']].to_latex(index=False, float_format='%.3f'))

In [ ]:
rows = duckdb.sql(
    " SELECT ratio, threads, epsilon, op, index_name, AVG(duration_sec) as v FROM results "
    " WHERE (ratio=10 or ratio=50 or ratio=100) AND input_size=200000000 AND op='join' AND index_type!='NA'"
    " AND (index_name='sj' OR epsilon=2048)"
    " GROUP BY ratio, threads, op, input_size, index_name, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
rows
print(duckdb.sql("PIVOT rows ON index_name USING AVG(v) GROUP BY threads,ratio ORDER BY threads,ratio").df().to_latex(index=False, float_format='%.3f'))

In [102]:
rows = duckdb.sql(
    " SELECT ratio, epsilon, op, index_type, AVG(duration_sec) as v FROM results "
    " WHERE threads = 1 AND input_size=200000000 AND op='merge' AND index_type!='NA'"
    " GROUP BY ratio, threads, op, input_size, index_type, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
p = (duckdb.sql("PIVOT rows ON index_type,epsilon USING avg(v) GROUP BY ratio ORDER BY RATIO"))
print(p)
print(p.df()[['ratio', 'SM_0', 'BTREE_256', 'BTREE_2048', 'PGM_256', 'PGM_2048']].to_latex(index=False, float_format='%.3f'))

┌───────┬─────────┬────────────────────┬────────────────────┬───┬────────────────────┬─────────┬─────────┬────────┐
│ ratio │ BTREE_0 │     BTREE_1024     │     BTREE_2048     │ … │        SM_0        │ SM_1024 │ SM_2048 │ SM_256 │
│ int64 │ double  │       double       │       double       │   │       double       │ double  │ double  │ double │
├───────┼─────────┼────────────────────┼────────────────────┼───┼────────────────────┼─────────┼─────────┼────────┤
│    10 │    NULL │  6.515239308055556 │  6.245084602555556 │ … │  7.306423629777778 │    NULL │    NULL │   NULL │
│    20 │    NULL │  4.709472161611111 │  4.545033750666666 │ … │  6.891521441611111 │    NULL │    NULL │   NULL │
│    30 │    NULL │ 4.1047253263888885 │ 3.9707228872777782 │ … │  6.734088810944444 │    NULL │    NULL │   NULL │
│    40 │    NULL │ 3.7468778676666665 │  3.696040477944444 │ … │  6.635971177333333 │    NULL │    NULL │   NULL │
│    50 │    NULL │       3.5406047055 │  3.518445066833333 │ … │  6.622

In [103]:
rows = duckdb.sql(
    " SELECT ratio, threads, epsilon, op, index_name, AVG(duration_sec) as v FROM results "
    " WHERE (ratio=10 or ratio=50 or ratio=100) AND input_size=200000000 AND op='merge' AND index_type!='NA'"
    " AND (index_name='standard_merge' OR epsilon=2048)"
    " GROUP BY ratio, threads, op, input_size, index_name, epsilon"
    " ORDER BY ratio, threads, epsilon"
)
rows
print(duckdb.sql("PIVOT rows ON index_name USING AVG(v) GROUP BY threads,ratio ORDER BY threads,ratio").df().to_latex(index=False, float_format='%.3f'))

\begin{tabular}{rrrr}
\toprule
threads & ratio & btree2048 & pgm2048 \\
\midrule
1 & 10 & 6.245 & 4.826 \\
1 & 50 & 3.518 & 3.243 \\
1 & 100 & 3.117 & 2.979 \\
4 & 10 & 2.702 & 2.647 \\
4 & 50 & 2.500 & 2.641 \\
4 & 100 & 2.532 & 2.436 \\
16 & 10 & 2.722 & 2.705 \\
16 & 50 & 2.550 & 2.514 \\
16 & 100 & 2.509 & 2.491 \\
32 & 10 & 2.900 & 2.949 \\
32 & 50 & 2.703 & 2.767 \\
32 & 100 & 2.749 & 2.730 \\
\bottomrule
\end{tabular}



In [ ]:
def group_by_threads(op, ratio):
    rows = duckdb.sql(
        " SELECT ratio,index_name,threads, MEDIAN(duration) as v"
        f" FROM results where op='{op}' AND ratio={ratio}  AND index_type!='NA'"
        " GROUP BY dataset, ratio, index_name, threads "
        " ORDER BY Threads")
    return duckdb.sql('PIVOT rows ON index_name USING MEDIAN(v)')
ratios = [10, 50, 100]
for ratio in ratios:
    for op in ['join', 'merge']:
        group_by_threads(op, ratio).df().to_csv('_'.join([op, str(ratio), 'threadStudy.csv']))

## PGM Index Memory Study

In [ ]:
def compare_btree_pgm_memory(epsilon, thread, dataset_type):
        where = f"WHERE threads={thread} AND epsilon={epsilon} AND ratio=10 "
        if dataset_type != "all":
                where = where + f" AND dataset_type='{dataset_type}' "
        query = ("SELECT ratio, threads, algo, index_type, dataset, dataset_type,"
                "epsilon, MEAN(index_memory) as memory_mean, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon, dataset, dataset_type) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING MEAN(memory_mean) GROUP BY ratio,dataset").df()
        df['REL'] = (df['PGM']) / df['BTREE']
        return df

def compare_btree_pgm_build_duration(epsilon, thread, dataset_type):
        where = f"WHERE threads={thread} AND epsilon={epsilon} AND ratio=10 "
        if dataset_type != "all":
                where = where + f" AND dataset_type='{dataset_type}' "
        query = ("SELECT ratio, threads, algo, index_type, dataset, dataset_type,"
                "epsilon, MEAN(index_build_duration) as index_build_duration, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon, dataset, dataset_type) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING MEAN(index_build_duration) GROUP BY ratio,dataset").df()
        df['REL'] = (df['PGM']) / df['BTREE']
        return df
dict = []
dict.append({
        'epsilon':256, 
        'ALL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=256, thread=1, dataset_type='all')['PGM'].mean(),
        })
dict.append({
        'epsilon':1024, 
        'ALL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=1024, thread=1, dataset_type='all')['PGM'].mean(),
        })
dict.append({
        'epsilon': 2048, 
        'ALL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='SYNTHETIC')['REL'].mean(),
        'ACTUAL': compare_btree_pgm_memory(epsilon=2048, thread=1, dataset_type='all')['PGM'].mean(),
        })

print(pd.DataFrame(dict).to_latex(index=False))

dict = []
dict.append({
        'epsilon':256, 
        'ALL': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=256, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
dict.append({
        'epsilon':1024, 
        'ALL': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=1024, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
dict.append({
        'epsilon': 2048, 
        'ALL': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='all')['REL'].mean(),
        'REAL': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='REAL')['REL'].mean(),
        'SYNTHETIC': compare_btree_pgm_build_duration(epsilon=2048, thread=1, dataset_type='SYNTHETIC')['REL'].mean()
        })
print(pd.DataFrame(dict).to_latex(index=False))

## PGM vs TREE SJ same epsilon (Duration)

In [ ]:
def compare_btree_pgm_duration(epsilon, thread, dataset, op):
        where = f"WHERE threads={thread} AND op='{op}' AND (epsilon={epsilon} OR algo='sj' OR algo='standard_merge')"
        if dataset != "all":
                where = where + f"AND dataset='{dataset}'"
        query = ("SELECT ratio, threads, algo, index_type,"
                "epsilon, MEDIAN(duration) as duration_mean, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        print(subresult)
        df = duckdb.sql("PIVOT subresult ON index_type USING sum(duration_mean) GROUP BY ratio").df()
        print(df)
        baseline = 'SJ' if op == 'join' else 'SM'
        df['RATIO'] = df['ratio']
        if 'PGM' in df.columns:
                df['PGM_REL'] = (df['PGM']/ df[baseline])
        if 'BTREE' in df.columns:
                df['BTREE_REL'] = (df['BTREE']/ df[baseline])
        return df

def compare_pgm_bytes_fetched(epsilon, thread, dataset, op):
        where = f"WHERE threads={thread} AND op='{op}' AND (epsilon={epsilon} OR algo='sj' OR algo='standard_merge')"
        if dataset != "all":
                where = where + f"AND dataset='{dataset}'"
        query = ("SELECT ratio, threads, algo, index_type,"
                "epsilon, MEDIAN(inner_bytes_fetched) as median, FROM results " 
                + where +
                "GROUP BY (ratio, threads, algo, index_type, epsilon) "
                "ORDER BY ratio")
        subresult = duckdb.sql(query)
        df = duckdb.sql("PIVOT subresult ON index_type USING AVG(median) GROUP BY ratio").df()
        baseline = 'SJ' if op == 'join' else 'SM'
        df['RATIO'] = df['ratio']
        df['BTREE_REL'] = (df['BTREE']/ df[baseline])
        df['PGM_REL'] = (df['PGM']/ df[baseline])
        return df


In [ ]:

df = pd.DataFrame();
df['ratio'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['RATIO'])
df['t=1'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'])
df['t=4'] = (compare_btree_pgm_duration(epsilon=2048, thread=4, dataset='all', op='join')['PGM_REL'])
df['t=16'] = (compare_btree_pgm_duration(epsilon=2048, thread=16, dataset='all', op='join')['PGM_REL'])
df['t=32'] = (compare_btree_pgm_duration(epsilon=2048, thread=32, dataset='all', op='join')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='join')['RATIO'])
df['t=1'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'])
df['t=4'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=4, dataset='all', op='join')['PGM_REL'])
df['t=16'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=16, dataset='all', op='join')['PGM_REL'])
df['t=32'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=32, dataset='all', op='join')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['RATIO'])
df['t=1'] = (compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'])
df['t=4'] = (compare_btree_pgm_duration(epsilon=2048, thread=4, dataset='all', op='merge')['PGM_REL'])
df['t=16'] = (compare_btree_pgm_duration(epsilon=2048, thread=16, dataset='all', op='merge')['PGM_REL'])
df['t=32'] = (compare_btree_pgm_duration(epsilon=2048, thread=32, dataset='all', op='merge')['PGM_REL'])
print(df.to_latex(index=False))

df = pd.DataFrame();
df['ratio'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='merge')['RATIO'])
df['t=1'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'])
df['t=4'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=4, dataset='all', op='merge')['PGM_REL'])
df['t=16'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=16, dataset='all', op='merge')['PGM_REL'])
df['t=32'] = (compare_pgm_bytes_fetched(epsilon=2048, thread=32, dataset='all', op='merge')['PGM_REL'])
print(df.to_latex(index=False))



In [ ]:
f = compare_btree_pgm_duration
df = []
df.append({'epsilon': 256, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='join')['PGM_REL'].mean()})
df.append({'epsilon': 1024, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=1024, thread=1, dataset='all', op='join')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['PGM_REL'].mean()})

df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})
df.append({'epsilon': 1024, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=1024, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='join')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())

f = compare_btree_pgm_duration
df = []
df.append({'epsilon': 256, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})

df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())

In [ ]:
f = compare_btree_pgm_duration
df = []

df.append({'epsilon': 2048, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})

df.append({'epsilon': 512, 'index': 'PGM', 't': 1, 'value': compare_btree_pgm_duration(epsilon=512, thread=1, dataset='all', op='merge')['PGM_REL'].mean()})
df.append({'epsilon': 2048, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=2048, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})
df.append({'epsilon': 256, 'index': 'BTREE', 't': 1, 'value': compare_btree_pgm_duration(epsilon=256, thread=1, dataset='all', op='merge')['BTREE_REL'].mean()})

df = pd.DataFrame(df)
print(df.pivot(index='epsilon', columns='index', values='value').to_latex())